In [ ]:
import pickle 
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
import os
import torch

drug_smiles_file='../data/223drugs_pubchem_smiles.txt'
save_dir = '../data/drug_embeddings'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

def store_embeddings(drug_embeddings, save_dir, model_name):
    with open(os.path.join(save_dir, f'{model_name}.pkl'), 'wb') as f:
        pickle.dump(drug_embeddings, f)

pubchemid2smile = {item.split('\t')[0]:item.split('\t')[1].strip() for item in open(drug_smiles_file).readlines()}

tokenizer = AutoTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
model = AutoModelForMaskedLM.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
model.eval()

drug_embeddings = {}
for pubchem_id, smile in pubchemid2smile.items():
    inputs = tokenizer(smile, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs["logits"][0, 0, :].numpy()
    drug_embeddings[pubchem_id] = embeddings
    print(f"Processed {pubchem_id} with SMILES: {smile}")

store_embeddings(drug_embeddings, save_dir, 'chemberta')
